[Reference](https://towardsdatascience.com/conquer-retries-in-python-using-tenacity-an-in-depth-tutorial-3c98b216d798)

In [6]:
!python -m pip install tenacity

In [1]:
import asyncio
import logging
import random
import sys

import aiohttp
from aiohttp import ClientTimeout, ClientSession
from tenacity import *


def ready_logger(stream, level) -> logging.Logger:
    logger = logging.getLogger(__name__)
    logger.setLevel(level)

    handler = logging.StreamHandler(stream)
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    return logger


logger = ready_logger(stream=sys.stderr, level=logging.DEBUG)


def my_before_log(logger: logging.Logger):
    def log_it(retry_state: RetryCallState):
        fn = retry_state.fn
        attempt = retry_state.attempt_number

        if attempt > 2:
            logger.warning(f"Retrying method {fn.__name__} at the {attempt} attempt")
    return log_it

In [2]:
def check_status(retry_state: RetryCallState) -> bool:
    outcome: Future = retry_state.outcome
    if outcome.exception():
        return True
    return outcome.result() > 300

In [4]:
@retry(stop=stop_after_attempt(20),
       wait=wait_incrementing(start=1, increment=1, max=5),
       before=my_before_log(logger),
       retry=check_status)
async def get_status(url_template: str, session: ClientSession) -> int:
    status_list = [200, 300, 400, 500]
    url = url_template.format(codes=random.choice(status_list))
    print(f"Begin to get status from {url}")
    async with session.get(url) as response:
        return response.status


async def main():
    timeout: ClientTimeout = aiohttp.ClientTimeout(2)
    async with aiohttp.ClientSession(timeout=timeout) as session:
        tasks = [asyncio.create_task(
            get_status('https://httpbin.org/status/{codes}', session)) for _ in range(5)]

        result = await asyncio.gather(*tasks)
        print(result)


if __name__ == "__main__":
    asyncio.run(main())